In [15]:
# Environment: google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install d2l==0.13.2 -f https://d2l.ai/whl.html # installing d2l
!pip install -U mxnet-cu101mkl==1.6.0  # updating mxnet to at least v1.6


Looking in links: https://d2l.ai/whl.html
     |████████████████████████████████| 711.0MB 19kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
%matplotlib inline
from d2l import mxnet as d2l
from mxnet import autograd, gluon, image, init, np, npx
from mxnet.gluon import nn

npx.set_np()

In [18]:
%cd /content/drive/My\ Drive/course/K131/StrayAnimalsAnalysis/imgs/raw_images
!ls

Streaming output truncated to the last 5000 lines.
106457_in.JPG.gz	  150307_front_org.JPG.gz  61515_front.jpg.gz
106460_front_org.jpg.gz   150308_front_org.JPG.gz  61539_in_org.JPG.gz
106466_in_org.JPG.gz	  150311_front.JPG.gz	   61556_front_org.JPG.gz
106470_front.JPG.gz	  150319_front.jpg.gz	   61561_in_org.JPG.gz
106475_front_org.JPG.gz   150320_front_org.jpg.gz  61570_front_org.jpg.gz
106481_front.JPG.gz	  150322_front_org.jpg.gz  61571_in_org.JPG.gz
106481_in_org.JPG.gz	  150328_front_org.jpg.gz  61574_in.jpg.gz
106484_front_org.JPG.gz   150328_in_org.jpg.gz	   61585_front_org.JPG.gz
106487_in.jpg.gz	  150333_in.jpg.gz	   61624_in_org.JPG.gz
106498_front.JPG.gz	  150353_in.JPG.gz	   61628_front_org.jpg.gz
106499_front_org.JPG.gz   150360_front.jpg.gz	   61638_in.jpg.gz
106507_in.JPG.gz	  150365_in.JPG.gz	   61640_front_org.JPG.gz
106510_in_org.JPG.gz	  150369_in_org.jpg.gz	   61645_front_org.jpg.gz
106513_front_org.JPG.gz   150370_front.jpg.gz	   61647_front.jpg.gz
106523_front_o